## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import optuna
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import lightgbm as lgb

### Data Load

In [2]:
# 데이터 로드
Total_train = pd.read_csv('../data/Total_train_dataset_30_copy.csv')
Total_test = pd.read_csv('../data/Total_test_dataset_30_copy.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
Total_X = Total_train.drop(['임신_성공_여부', 'ID'], axis=1)
Total_y = Total_train['임신_성공_여부']

### 인코딩 

In [4]:
Total_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "배란_유도_유형",
    "단일_배아_이식_여부",
    "착상_전_유전_진단_사용_여부",
    "배아_생성_주요_이유",
    "총_생성_배아_수",
    "미세주입된_난자_수",
    "미세주입에서_생성된_배아_수",
    "이식된_배아_수",
    "미세주입_배아_이식_수",
    "저장된_배아_수",
    "미세주입_후_저장된_배아_수",
    "해동된_배아_수",
    "해동_난자_수",
    "수집된_신선_난자_수",
    "저장된_신선_난자_수",
    "혼합된_난자_수",
    "파트너_정자와_혼합된_난자_수",
    "기증자_정자와_혼합된_난자_수",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "동결_배아_사용_여부",
    "신선_배아_사용_여부",
    "기증_배아_사용_여부",
    "대리모_여부",
    "ICSI_배아_이식_비율",
    "ICSI_배아_생성_비율",
    "ICSI_성공률"
]

In [5]:
# 모든 범주형 변수를 문자열로 변환
Total_X[Total_categorical_columns] = Total_X[Total_categorical_columns].astype(str)
Total_test[Total_categorical_columns] = Total_test[Total_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
Total_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

Total_X[Total_categorical_columns] = Total_encoder.fit_transform(Total_X[Total_categorical_columns])
Total_test[Total_categorical_columns] = Total_encoder.transform(Total_test[Total_categorical_columns])

## Modeling

In [6]:
# 데이터 분할
Total_X_train, Total_X_test, Total_y_train, Total_y_test = train_test_split(Total_X, Total_y, test_size=0.2, random_state=42)

### Total 데이터

optuna

In [7]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 1000, 8000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True),
        'depth': trial.suggest_int('depth', 2, 16),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-4, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
        'border_count': trial.suggest_int('border_count', 1, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 10.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),

        'boosting_type': 'Plain',
        
        'random_state': 42,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'thread_count': -1
    }

    model = CatBoostClassifier(**param)
    model.fit(Total_X_train, Total_y_train, eval_set=(Total_X_test, Total_y_test), verbose=False)
    
    y_pred_proba = model.predict_proba(Total_X_test)[:, 1]
    
    auc = roc_auc_score(Total_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1200)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-14 23:48:31,377] A new study created in memory with name: no-name-6be205b5-1880-4fa2-8886-425f77896b4b
[I 2025-02-15 00:00:07,378] Trial 0 finished with value: 0.7231358426437665 and parameters: {'iterations': 4384, 'learning_rate': 5.389427875463091e-06, 'depth': 8, 'min_data_in_leaf': 3, 'l2_leaf_reg': 0.0001102030793969428, 'random_strength': 3.750874028992066, 'bagging_temperature': 5.023566258260629, 'border_count': 137, 'scale_pos_weight': 3.431920556272233, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 0.7231358426437665.
[I 2025-02-15 00:01:59,660] Trial 1 finished with value: 0.7203404036655595 and parameters: {'iterations': 1693, 'learning_rate': 0.002658259243797894, 'depth': 4, 'min_data_in_leaf': 31, 'l2_leaf_reg': 0.003941930727562303, 'random_strength': 6.108264843944272, 'bagging_temperature': 5.979192482987826, 'border_count': 146, 'scale_pos_weight': 4.126379291682463, 'grow_policy': 'Lossguide'}. Best is trial 0 with value: 0.7231358426437665.
[

KeyboardInterrupt: 

.